# Feature Analysis Statistics
Provides various statistics about features in the context of a binary classifier. The study is limited to transcription factors (reglators).

In [1]:
import init
import common.constants as cn
import common_python.constants as ccn
from common.trinary_data import TrinaryData
from common.data_provider import DataProvider
import classifier.main_multi_classifier_feature_optimizer as main
from common.data_provider import DataProvider
from common_python.plots import util_plots
from plots import util_plots as xutil_plots
from common_python.classifier.feature_analyzer import FeatureAnalyzer

import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn

%matplotlib inline

In [2]:
# These are the "long" data that have individual replications, not averages. And, only using TFs.
PROVIDER = DataProvider()
PROVIDER.do()
TRINARY = TrinaryData(is_averaged=False, is_dropT1=False, is_regulator=True)  # Trinary data
DF_X = TRINARY.df_X
SER_Y = TRINARY.ser_y
STATES = SER_Y.unique()
REGULATORS = DF_X.columns.tolist()
CLF = svm.LinearSVC()
ANALYZER = FeatureAnalyzer(CLF, DF_X, SER_Y, num_cross_iter=50)

## Single Feature Accuracy (SFA)

In [ ]:
# For each state, plot the accuracy of a single regulator classifier
